In [1]:
import sys
import nibabel as nib
import os
import numpy as np
from datetime import datetime
from glob import glob
from natsort import natsorted
import itertools
import skimage
from skimage import segmentation
from skimage import measure
from matplotlib import pyplot as plt
from scipy.spatial import distance
from skimage import segmentation
from pprint import pprint
import csv

In [2]:
# define paths

# paths to original GIF output
parc_paths = natsorted(glob(os.path.join("parcellation_data", "GIF_with_tumour_9classes", "vs_gk*", "vs_gk_seg_GIF_multiclass_refT1.nii.gz")))
output_foldername = os.path.join("random_forest_classification", "features", "extracted_from_GIF_output")

# # paths to nnU-Net output trained with T1
# parc_paths = natsorted(glob(os.path.join("parcellation_data", "GIF_UNet_prediction_9_classes_T1", "vs_gk*", "vs_gk_seg_GIF_multiclass_mergedCerebellum_refT1.nii.gz")))
# output_foldername = os.path.join("random_forest_classification", "features", "extracted_from_nnUNet_prediction", "T1")

# # paths to nnU-Net output trained with T2
# parc_paths = natsorted(glob(os.path.join("parcellation_data", "GIF_UNet_prediction_9_classes_T2", "vs_gk*", "vs_gk_seg_GIF_multiclass_mergedCerebellum_refT1.nii.gz")))
# output_foldername = os.path.join("random_forest_classification", "features", "extracted_from_nnUNet_prediction", "T2")

# # paths to nnU-Net output trained with T1T2
# parc_paths = natsorted(glob(os.path.join("parcellation_data", "GIF_UNet_prediction_9_classes_T1T2", "vs_gk*", "vs_gk_seg_GIF_multiclass_mergedCerebellum_refT1.nii.gz")))
# output_foldername = os.path.join("random_forest_classification", "features", "extracted_from_nnUNet_prediction", "T1T2")

In [3]:
# name the labels that are present in the segmentations
label_dict = {
 '0': 'Background',
 '1':'Tumour',
 '2':'Pons',
 '3':'Brainstem',
 '4':'Cerebellar Vermal Lobules I-V',
 '5':'Cerebellar Vermal Lobules VI-VII',
 '6':'Cerebellar Vermal Lobules VIII-X',
 '7':'Right Cerebellum',
 '8':'Left Cerebellum',
 }

In [4]:
# function to get the neighbouring voxels of voxel with coordinates x,y,z
def get_neighbors(x, y, z):
    neighbors = []
    for i,j,k in [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]: 
        neighbors.append([x-i, y-j, z-k])
        
    return neighbors

In [5]:
for parc_path in parc_paths:

    parc_nii = nib.load(parc_path)
    parc_data = np.array(parc_nii.get_fdata(), dtype=int)
    spacing = parc_nii.header['pixdim'][1:4]

    # get unique values
    label_number_list = np.unique(parc_data)

    tumour_label_number = 1

    ## get boundary pixels of tumour and set the value to the same label as neighbor
    border_pixels_img = np.ones_like(parc_data)*(-1)

    tumour_voxel_inds = np.where(parc_data == tumour_label_number)
    for x, y, z in zip(*tumour_voxel_inds):

        if parc_data[x,y,z] == tumour_label_number:
            neighbors = get_neighbors(x,y,z)
            for nei in neighbors:
                n = int(parc_data[tuple([[c] for c in nei])])
                if n !=tumour_label_number:
                    border_pixels_img[x,y,z] = n


    ## calculate surface between tumour and each other tissue
    tumour_surface_voxel_inds = np.where(border_pixels_img != -1)
    present_neighbour_labels = np.unique(border_pixels_img)

    print("Calculate smallest distance btween tumour and parcellations")
    surface_areas = {n:0 for n in present_neighbour_labels}
    for n in present_neighbour_labels:
        binary_3d_surf_data = np.zeros_like(border_pixels_img)
        binary_3d_surf_data[border_pixels_img==n] = 1

        # Use marching cubes to obtain the surface mesh
        verts, faces, normals, values = measure.marching_cubes(binary_3d_surf_data, level=0, spacing=spacing)

        surface_areas[n] = skimage.measure.mesh_surface_area(verts, faces)


    ## calculate smallest distance btween tumour and parcellations
    dists = {}

    tumour_coords = list(zip(*tumour_voxel_inds))

    print("Iterate over parcels and calculate minimum distance to VS for each parcel...")
    for n in label_number_list:
        print("current label:", n)
        # extract only the voxels on the surface of the nth parcellation
        single_parc_binary_img = parc_data==n
        boundaries = skimage.segmentation.find_boundaries(single_parc_binary_img, connectivity=1, mode='inner', background=0)

        single_parc_surf_voxel_inds = list(zip(*np.where(boundaries)))

        # for parcels with many voxels, the resulting distance matrix of scipy.spatial.distance.cdist does not fit in memory
        # therefore, define the maximum matrix size, split up larger matrices and iterate over them
        max_matrix_elem_in_memory = 776421036  # maximum matrix size
        steps=int(np.ceil(len(tumour_coords)*len(single_parc_surf_voxel_inds)/max_matrix_elem_in_memory))  # number of parts into which parcel is split
        max_elem_in_parc_vector = int(np.ceil(len(single_parc_surf_voxel_inds)/(steps)))  # length of one sub-parcel

        dist = np.Inf
        for s in range(steps): # iterate over sub-parcels and save only the minimum distance
            part_of_single_parc_surf_voxel_inds = single_parc_surf_voxel_inds[s*max_elem_in_parc_vector:(s+1)*max_elem_in_parc_vector]
            min_dist = np.min(distance.cdist(np.multiply(tumour_coords, spacing), np.multiply(part_of_single_parc_surf_voxel_inds, spacing), 'euclidean'))
            dist = min_dist if min_dist < dist else dist
        dists[n] = dist


    ## calculate volumes of all parcels
    vols = {}
    for n in label_number_list:
        nb_voxels = np.count_nonzero(parc_data==n)
        vols[n] = nb_voxels*np.prod(spacing)

    ## collect all features
    features = {n:{'name':'0', 'vol':0, 'surfVS':0, 'distVS':np.nan} for n in label_number_list}
    for n in label_number_list:

        features[n]['vol'] = vols[n]
        features[n]['name'] = label_dict[str(n)]
        if n in surface_areas:
            features[n]['surfVS'] = surface_areas[n]
        if n in dists:
            features[n]['distVS'] = dists[n]

    ## print all features
    print(f"Features case {n}:")
    col_width = 50
    # Print the names of the columns.
    print ("label\t"+("{:<{col_width}} "*len(list(features.values())[0].keys())).format(*list(features.values())[0].keys(), col_width=col_width))

    # print each data item.
    for key, value in features.items():
        #name, vol, surfVS = *value.values()
        print (str(key)+"\t"+("{:<{col_width}} "*len(value.keys())).format(*value.values(), col_width=col_width))

    ##save features as csv
    for key in features:
        features[key]["labelnumber"] = key

    csv_columns = list(list(features.values())[0].keys())
    case_name = parc_path.split(os.sep)[-2]
    csv_file = os.path.join(output_foldername, case_name+".csv")

    if not os.path.isdir(os.path.dirname(csv_file)):
        os.makedirs(os.path.dirname(csv_file), exist_ok=True)

    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["labelnumber", "name", "vol", "surfVS", "distVS"])
            writer.writeheader()
            for data in list(features.values()):
                writer.writerow(data)
    except IOError:
        print("I/O error")

Calculate smallest distance btween tumour and parcellations
Iterate over parcels and calculate minimum distance to VS for each parcel...
current label: 0
current label: 1
current label: 2
current label: 3
current label: 4
current label: 5
current label: 6
current label: 7
current label: 8
Features case 8:
label	name                                               vol                                                surfVS                                             distVS                                             
0	Background                                         3378828.567123413                                  646.3290405273438                                  0.41015625                                         
1	Tumour                                             483.7400436401367                                  0                                                  0.0                                                
2	Pons                                               8614.9635314941